In [1]:
import pickle
import numpy as np
from s3_train import build_model
from utils import MAX_LENGTH, LABEL_DICT

/home/qhduan/Work/cpu/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
inverse_ld = {v: k for k, v in LABEL_DICT.items()}

In [3]:
data = pickle.load(open('data.pkl', 'rb'))

In [4]:
ws = pickle.load(open('ws.pkl', 'rb'))

In [5]:
shape = (MAX_LENGTH,)

In [6]:
model, atten_model = build_model(shape, len(ws))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 64)      648576      input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 200, 64)      0           embedding_1[0][0]                
________________________________________________________________________________________

In [7]:
model.load_weights('./model_gru_weights.hdf5')

In [12]:
def test(sentence):
    vec = ws.transform(sentence, MAX_LENGTH)
#     print(vec)
    x = np.array([vec])
    pred = model.predict(x)
    atten = atten_model.predict(x)
    atten = atten.reshape((MAX_LENGTH, 32))
    atten = np.sum(atten, axis=-1)
    print(atten.shape)
    
    ret = []
    for i in range(len(LABEL_DICT)):
        ret.append((
            inverse_ld[i],
            pred[0][i] * 100
        ))
    ret = sorted(ret, key=lambda x: x[1], reverse=True)
    for r in ret:
        print('{}\t\t{:.2f}%'.format(
            r[0],
            r[1]
        ))
#     for i, w in enumerate(list(sentence)):
#         print(atten[i], w)

In [13]:
def random_test(data, max_length=MAX_LENGTH):
    while True:
        ind = np.random.randint(0, len(data))
        x = data[ind][0]
        y = data[ind][2]
        if len(x) <= max_length:
            break
    print(x)
    print(y)
    test(x)

In [15]:
for i in range(20):
    random_test(data)

戴老师所言极是，我家每天的米饭都是大米，玉米渣，高粱米，荞麦米，小黄米，紫米搭配的，另外再加点紫薯块和南瓜块，老公天天说，想吃点白米饭怎么那么难呢，，其实这样搭配的另外一个好处就是，粗粮，饱腹感很强，可以抑制食欲
happy
(200,)
happy		30.01%
disgust		22.97%
angry		21.95%
sad		10.85%
fear		9.62%
suprise		4.60%
各种笑容的英文表达，smile（微笑）、laugh（大笑）、grin（露齿而笑）、chuckle（轻笑）、smirk（得意地笑）、simper（痴笑）、snicker（坏笑）、giggle（咯咯笑）、titter（）、guffaw（狂笑）、roar（哄笑）、chortle（欢笑）、ridicule（嘲笑）、deride（讥笑）
happy
(200,)
happy		88.83%
disgust		4.43%
fear		2.33%
sad		2.01%
angry		1.74%
suprise		0.67%
爲了兩件垃圾系度同我講錢仲要問候埋我娘親求你都系多餘，個晚擔心你安危都系多7魚，仲講乜野朋友我呸呸甘多年10年我都未講耐1000蚊買起你人格，講錢傷感情，講到錢乜野兄弟朋友都無得做，全部通通走開，全都爲個區區臭錢，從今開始互不借錢，終於明白咩叫借錢如送禮，還錢如乞米
angry
(200,)
angry		69.69%
disgust		10.22%
sad		8.71%
suprise		4.87%
fear		4.67%
happy		1.84%
很久没看電視，今晚看到碧X洗衣液的廣告，還＂粉絲鄰居＂呢，配合小S＂不裝B會死＂的表情，感覺真賤啊
happy
(200,)
happy		45.63%
disgust		17.55%
angry		14.33%
sad		10.81%
fear		8.83%
suprise		2.86%
回复这是我自己猜猜的，说是超级大腕律师，肯定非同一般，又是专业律师，又安排元月四号讲座，再结合往年的建纬与大成的接触，可想而知的。我估计就是他。你现在可以去大成挖一个更大的过来，比如把挖过来，，刘不是要辞去主任啦？机会啊 。
happy
(200,)
happy		45.16%
disgust		20.94%
fear		12.